In [16]:
import os
from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, BaseMessage
from langchain_experimental import tot
from typing import List
import json
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.schema.runnable import RunnablePassthrough

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=openai_api_key,
    streaming=True,
    temperature=0.0,
)

class MethodDetail(BaseModel):
    reasoning: str
    passages: List[str]
    confidence_score: float

class MethodOutput(BaseModel):
    methods: List[str]
    method_details: dict[str, MethodDetail]
    
method_parser = JsonOutputParser(pydantic_object=MethodOutput)
system_prompt = SystemMessage(
    content="""
    You are a method extraction AI whose purpose is to identify and extract method keywords from an academic abstract. Your role is to locate the specific methodologies, techniques, or approaches mentioned in the abstract and provide justification for why each keyword represents a method.

    ### Definition of Methods:
    - "Methods" refers to the **specific processes**, **techniques**, **procedures**, or **approaches** used in conducting the research. This includes techniques for data collection, data analysis, algorithms, experimental procedures, or any other specific methodology employed by the researchers. Methods should not include general descriptions, conclusions, or research themes.

    ### What You Should Do:
    1. Extract keywords that refer to the **methods** used in the abstract.
    2. For each keyword, provide a **reasoning** explaining why it represents a method in the context of the abstract.
    3. Present the results in the required **JSON format** with a list of methods and justifications for each.

    ### JSON Output Requirements:
    - **Response Format**: You must return your output as a JSON object.
    - The JSON object must contain:
    - A key `"methods"` whose value is a list of extracted **method keywords**.
    - A key for each method keyword that containes 2 keys:
        - `"reasoning"`: A string that provides the **reasoning** behind why that keyword was extracted.
        - "passages": A list of strings that are the passages from the abstract that lead you to believe that this is a method keyword.
        - "confidence_score": A float between 0 and 1 that represents the confidence in the keyword.
        
    ### JSON Structure:
    ```json
        {
        "methods": [
            "<method_keyword_1>",
            "<method_keyword_2>"
        ],
        "<method_keyword_1>": {
            "reasoning": "<explain why this is a method keyword>",
            "passages": ["<list of passages from the abstract which lead you to believe this is a method keyword>"],
            "confidence_score": <confidence score float value between 0 and 1>
        },
        "<method_keyword_2>": {
            "reasoning": "<explain why this is a method keyword>"
            "passages": ["<list of passages from the abstract which lead you to believe this is a method keyword>"],
            "confidence_score": <confidence score float value between 0 and 1>
        }
    }
    ```
    
    See the following examples:
    
    ### Example 1: Correct Extraction

    **Abstract:**
    “Drawing on expectation states theory and expertise utilization literature, we examine the effects of team members’ actual expertise and social status on the degree of influence they exert over team processes via perceived expertise. We also explore the conditions under which teams rely on perceived expertise versus social status in determining influence relationships in teams. To do so, we present a contingency model in which the salience of expertise and social status depends on the types of intragroup conflicts. Using multiwave survey data from 50 student project teams with 320 members at a large national research institute located in South Korea, we found that both actual expertise and social status had direct and indirect effects on member influence through perceived expertise. Furthermore, perceived expertise at the early stage of team projects is driven by social status, whereas perceived expertise at the later stage of a team project is mainly driven by actual expertise. Finally, we found that members who are being perceived as experts are more influential when task conflict is high or when relationship conflict is low. We discuss the implications of these findings for research and practice.”

    Output:
    ```json
    {
        "methods": [
            "multiwave survey data collection",
            "contingency modeling"
        ],
        "multiwave survey data collection": {
            "reasoning": "Multiwave survey data collection is the specific method used to gather data from participants over multiple time points, providing a clear methodological process for the research.",
            "passages": [
                "Using multiwave survey data from 50 student project teams with 320 members at a large national research institute located in South Korea"
            ],
            "confidence_score": 0.95
        },
        "contingency modeling": {
            "reasoning": "Contingency modeling is the method used to analyze the relationship between expertise, social status, and intragroup conflicts, forming the backbone of the data analysis.",
            "passages": [
                "we present a contingency model in which the salience of expertise and social status depends on the types of intragroup conflicts"
            ],
            "confidence_score": 0.90
        }
    }
    ```
    
    #### Explanation for Correct Extraction:
    
    - **Multiwave survey data collection**: This is a method because it refers to how data was gathered from the research subjects over multiple time points. The **confidence score (0.95)** reflects that this is a well-established data collection method.
    - **Contingency modeling**: This is a method because it describes the analytical process used to explore relationships between variables like expertise and social status. The **confidence score (0.90)** reflects the significance of this method in the research.
    
    ### Example 2: Incorrect Extraction

    **Abstract:**
    “Drawing on expectation states theory and expertise utilization literature, we examine the effects of team members’ actual expertise and social status on the degree of influence they exert over team processes via perceived expertise. We also explore the conditions under which teams rely on perceived expertise versus social status in determining influence relationships in teams. To do so, we present a contingency model in which the salience of expertise and social status depends on the types of intragroup conflicts. Using multiwave survey data from 50 student project teams with 320 members at a large national research institute located in South Korea, we found that both actual expertise and social status had direct and indirect effects on member influence through perceived expertise. Furthermore, perceived expertise at the early stage of team projects is driven by social status, whereas perceived expertise at the later stage of a team project is mainly driven by actual expertise. Finally, we found that members who are being perceived as experts are more influential when task conflict is high or when relationship conflict is low. We discuss the implications of these findings for research and practice.”
    
    Output:
    ```json
    {
        "methods": [
            "intragroup conflict",
            "perceived expertise",
            "social status",
            "multiwave survey data collection"
        ],
        "intragroup conflict": {
            "reasoning": "Intragroup conflict is a key factor in determining team dynamics and was analyzed in the research.",
            "passages": [
                "the salience of expertise and social status depends on the types of intragroup conflicts"
            ],
            "confidence_score": 0.75
        },
        "perceived expertise": {
            "reasoning": "Perceived expertise is one of the core variables examined in the study, making it a methodological focus.",
            "passages": [
                "perceived expertise at the early stage of team projects is driven by social status"
            ],
            "confidence_score": 0.70
        },
        "social status": {
            "reasoning": "Social status is an important factor that influences member dynamics in teams, making it a key methodological focus.",
            "passages": [
                "perceived expertise at the early stage of team projects is driven by social status"
            ],
            "confidence_score": 0.65
        },
        "multiwave survey data collection": {
            "reasoning": "Multiwave survey data collection is the method used to gather data from participants over multiple time points, providing a clear methodological process for the research.",
            "passages": [
                "Using multiwave survey data from 50 student project teams with 320 members at a large national research institute located in South Korea"
            ],
            "confidence_score": 0.95
        }
    }
    ```
    
    #### Explanation for Incorrect Extraction:

    - **Intragroup conflict**: This is incorrect because **intragroup conflict** is a variable or condition examined in the research, not a method. It is part of the analysis, not a process or technique used to conduct the research.
    - **Perceived expertise**: This is incorrect because **perceived expertise** is a measured variable, not a method. It’s what the study investigates, but it’s not a methodological process.
    - **Social status**: This is incorrect because **social status** is another variable the study looks at. Like the others, it’s part of the analysis, not a method.
    
    IMPORTANT: Do not include the markdown json code block notation in your response. Simply return the JSON object.
    The markdown json code block notation is: ```json\n<your json here>\n```, do not include the ```json\n``` in your response.
    IMPORTANT: You must return the output in the specified JSON format. If you do not return the output in the specified JSON format, you have failed.
    """
)


abstract = """
    Drawing on expectation states theory and expertise utilization literature, we examine the effects of team members' actual expertise and social status on the degree of influence they exert over team processes via perceived expertise. We also explore the conditions under which teams rely on perceived expertise versus social status in determining influence relationships in teams. To do so, we present a contingency model in which the salience of expertise and social status depends on the types of intragroup conflicts. Using multiwave survey data from 50 student project teams with 320 members at a large national research institute located in South Korea, we found that both actual expertise and social status had direct and indirect effects on member influence through perceived expertise. Furthermore, perceived expertise at the early stage of team projects is driven by social status, whereas perceived expertise at the later stage of a team project is mainly driven by actual expertise. Finally, we found that members who are being perceived as experts are more influential when task conflict is high or when relationship conflict is low. We discuss the implications of these findings for research and practice.
    """

first_prompt = PromptTemplate(
    template="{system_prompt}\n\nAbstract:\n{abstract}\n",
    input_variables=["system_prompt.content", "abstract"],
    partial_variables={"format_instructions": method_parser.get_format_instructions()}
)

chain = first_prompt | llm | method_parser

output = chain.invoke({"system_prompt": system_prompt.content, "abstract": abstract})
print(output)
methods = output["methods"]
# Abstract sentence by setence
class SentenceDetails(BaseModel):
    sentence: str
    meaning: str
    reasoning: str
    confidence_score: float

class AbstractThemes(BaseModel):
    sentence_details: List[SentenceDetails]
    overall_theme: str
    summary: str
    
abstract_parser = JsonOutputParser(pydantic_object=AbstractThemes)

methods = ", ".join(output["methods"])
setence_analysis_json_example = """
    {
      "sentence_details": [
        {
          "sentence": "Original sentence 1",
          "meaning": "Meaning of the sentence.",
          "reasoning": "Why this is the meaning of the sentence.",
          "confidence_score": Confidence score (0.0 - 1.0)
        },
        {
          "sentence": "Original sentence 2",
          "meaning": "Meaning of the sentence.",
          "reasoning": "Why this is the meaning of the sentence.",
          "confidence_score": Confidence score (0.0 - 1.0)
        },
        ...
      ],
      "overall_theme": "Overall theme of the abstract",
      "summary": "Detailed summary of the abstract"
    }
"""

abstract_prompt_template = """
    You are tasked with analyzing an abstract of a research paper. Your task involves the following steps:

    Steps to follow:
    1. **Record each sentence in the abstract**: 
    2. For each sentence do the following steps: 
    - Determine the meaning of the sentence
    - Provide a reasoning for your interpretation
    - Assign a confidence score between 0 and 1 based on how confident you are in your assessment.
    3. After you have done this for each sentence, determine the overall theme of the abstract. This should be a high-level overview of the main idea of the research.
    4. Provide a detailed summary of the abstract. This should be a thorough overview of the research, including the main idea, the methods used, and the results.
       
    Your output should follow this structure:

    {setence_analysis_json_example}

    IMPORTANT: Be concise but clear in your meanings and reasonings.
    IMPORTANT: Ensure that the confidence score reflects how certain you are about the meaning of the sentence in context.
    IMPORTANT: Do not include the markdown json code block notation in your response. Simply return the JSON object. The markdown json code block notation is: ```json\n<your json here>\n```, do not include the ```json\n``` in your response.
    IMPORTANT: You must return the output in the specified JSON format. If you do not return the output in the specified JSON format, you have failed.
    """

abstract_system_prompt = SystemMessage(
    content=abstract_prompt_template,
    input_variables=["setence_analysis_json_example"]
)

abstract_prompt = PromptTemplate(
   template="{abstract_system_prompt}\n\n## Abstract: \n{abstract}\n",
   input_variables=["abstract_system_prompt.content", "abstract"],
   partial_variables={"format_instructions": abstract_parser.get_format_instructions()}
)

json_structure = """
    {
        "summary": "Detailed summary of the abstract",
        "reasoning": "Detailed reasoning for the summary",
        "feedback": {
            "methodologies_feedback": "Feedback for the methodologies assistant",
            "abstract_sentence_analysis_feedback": "Feedback for the abstract sentence analysis assistant"
        }
    }
"""

class Feedback(BaseModel):
    assistant_name: str
    feedback: str

class AbstractSummary(BaseModel):
    summary: str
    reasoning: str
    feedback: List[Feedback]
    
abstract_summary_parser = JsonOutputParser(pydantic_object=AbstractSummary)

abstract_chain = abstract_prompt | llm | abstract_parser
abstract_chain_output = json.dumps(abstract_chain.invoke({"abstract_system_prompt": abstract_system_prompt.content, "abstract": abstract,}), indent=4)
method_json_format = """
    {
        "methods": [
            "<method_keyword_1>",
            "<method_keyword_2>"
        ],
        "<method_keyword_1>": {
            "reasoning": "<explain why this is a method keyword>",
            "passages": ["<list of passages from the abstract which lead you to believe this is a method keyword>"],
            "confidence_score": <confidence score float value between 0 and 1>
        },
        "<method_keyword_2>": {
            "reasoning": "<explain why this is a method keyword>"
            "passages": ["<list of passages from the abstract which lead you to believe this is a method keyword>"],
            "confidence_score": <confidence score float value between 0 and 1>
        }
    }
    """
method_json_output = json.dumps(output, indent=4)

json_structure = """
    {
        "summary": "Detailed summary of the abstract",
        "reasoning": "Detailed reasoning for the summary",
        "feedback": {
            "methodologies_feedback": "Feedback for the methodologies assistant",
            "abstract_sentence_analysis_feedback": "Feedback for the abstract sentence analysis assistant"
        }
    }
"""

abstract_summary_system_template = PromptTemplate(
    template="""
    You are an expert AI researcher that is tasked with summarizing academic research abstracts. Your task is to analyze the abstract and extract the main ideas and themes. You should not use the identified methods to form this summary, this summary should focus on the main idea of the research, as in what the research is doing rather than how it is doing it.

    In order to better assist you the following data will be provided:
    
    1. **Methodologies:**
    
    A former AI assistant has already extracted the methodologies from the abstract. It provides the identified methodologies. For each methodology it provides reasoning for why it identified it as a methodology, the passage(s) from the abstract which support its identification as a methodology, and a confidence score for its identification as a methodology. Here are the methodologies identified by the previous assistant:
        
    Here is the format the output from the methdologies assistant is in:
    ```json
    {method_json_format}
    ```
    
    2. **Abstract Sentence Level Analysis:**
    
    Another previous assistant has already analyzed each sentence in the abstract. For each sentence is provides the identified meaning, the reasoning why they identified that meaning, and a confidence score for the identified meaning. It also provides an overall theme of the abstract and a detailed summary of the abstract.
    
    Here is the format the output from the abstract sentence level analysis assistant is in, this format example is annotated so you can understand what each element is:
    ```json
    {setence_analysis_json_example}
    ```
    
    Output from the methodologies assistant:
    ```json
    {method_json_output}
    ```

    Output from the abstract sentence level analysis assistant:
    ```json
    {abstract_chain_output}
    ```

    Your output should contain the following:
    - summary: A detailed summary of the abstract which aims to capture the main idea of the research while not being concerned with the specific methods used to conduct the research.
    - reasoning: A detailed reasoning for the summary you have provided.
    - feedback for the methodologies assistant: Feedback detailing any issues you may think of that may have affected your ability to accurately summarize the abstract, as well as any requests you may have for the previous assistant to improve their analysis of the abstract so that you can more easily summarize it. Be as specific as possible. Do not provide feedback for the sake of providing feedback, provide feedback that will actually help the methodologies assistant improve their analysis of the abstract. If you believe the analysis is correct and you have no feedback, simply provide "The analysis is correct and sufficient, I have no feedback at this time." and nothing else.
    - feedback for the abstract sentence level analysis assistant: Feedback detailing any issues you may think of that may have affected your ability to accurately summarize the abstract, as well as any requests you may have for the previous assistant to improve their analysis of the abstract so that you can more easily summarize it. Be as specific as possible. Do not provide feedback for the sake of providing feedback, provide feedback that will actually help the abstract sentence level analysis assistant improve their analysis of the abstract. If you believe the analysis is correct and you have no feedback, simply provide "The analysis is correct and sufficient, I have no feedback at this time." and nothing else.

    You should follow these steps to complete your task:
    1. Carefully read and understand the methodologies identified by the previous assistant.
    2. Carefully read and understand the sentence level analysis of the abstract provided by the previous assistant.
    3. Carefully read and understand the abstract as a whole.
    4. Form a detailed summary of the abstract which captures the main idea of the research while not being concerned with the specific methods used to conduct the research.
    5. Provide a detailed reasoning for the summary you have provided.
    6. Provide feedback for the methodologies assistant.
        - Carefully review your process to identify what you did well and what you could improve on and based on what you could improve on identify if there is anything the methodologies assistant could improve on in their analysis of the abstract.
    7. Provide feedback for the abstract sentence level analysis assistant.
        - Carefully review your process to identify what you did well and what you could improve on and based on what you could improve on identify if there is anything the abstract sentence level analysis assistant could improve on in their analysis of the abstract.
        
    Your ouput should be a JSON object with the following structure:
    ```json
    {json_structure}
    ```

    IMPORTANT: Your summary should focus on the main idea of the research while not being concerned with the specific methods used to conduct the research. If you are concerned with the specific methods used to conduct the research, you are doing it wrong. If you summary contains mentions of the methodologies used, you are doing it wrong.
    IMPORTANT: Ensure that your feedback is specific to the methodologies assistant and abstract sentence level analysis assistant. Do not provide feedback for the sake of providing feedback, provide feedback that will actually help the assistants improve their analysis of the abstract.
    IMPORTANT: Do not include the markdown json code block notation in your response. Simply return the JSON object. The markdown json code block notation is: ```json\n<your json here>\n```, do not include the ```json\n``` in your response.
    IMPORTANT: You must return the output in the specified JSON format. If you do not return the output in the specified JSON format, you have failed.
    """
)

# summary_system_prompt = SystemMessage(
#     content=abstract_summary_system_template,
#     input_variables=["method_json_output", "abstract_chain_output", "abstract"]
# )
    

# summary_prompt = PromptTemplate(
#   template="{abstract_summary_system_template}\n\n## Original Abstract: \n{abstract}",
#   input_variables=["abstract_summary_system_template.content", "abstract"],
#   partial_variables={"format_instructions": abstract_summary_parser.get_format_instructions()}
# )

# summary_chain = summary_prompt | llm | abstract_summary_parser
# summary_chain_output = summary_chain.invoke({"abstract_summary_system_template": summary_system_prompt.content, "abstract": abstract})
# print(json.dumps(summary_chain_output, indent=4))
# Print the summary, reasoning, and feedback

from langchain.schema.runnable import RunnablePassthrough, RunnableParallel
from langchain.prompts import ChatPromptTemplate

abstract_0 = """
    Drawing on expectation states theory and expertise utilization literature, we examine the effects of team members' actual expertise and social status on the degree of influence they exert over team processes via perceived expertise. We also explore the conditions under which teams rely on perceived expertise versus social status in determining influence relationships in teams. To do so, we present a contingency model in which the salience of expertise and social status depends on the types of intragroup conflicts. Using multiwave survey data from 50 student project teams with 320 members at a large national research institute located in South Korea, we found that both actual expertise and social status had direct and indirect effects on member influence through perceived expertise. Furthermore, perceived expertise at the early stage of team projects is driven by social status, whereas perceived expertise at the later stage of a team project is mainly driven by actual expertise. Finally, we found that members who are being perceived as experts are more influential when task conflict is high or when relationship conflict is low. We discuss the implications of these findings for research and practice.
    """

abstract_1 = """
    The goal of this paper is to investigate how deregulating foreign equity ownership influences a firm's innovation investment. We attempt to answer this question using data from 530 Korean manufacturing firms between 1998 and 2003 through generalised estimating equations. Our findings suggest that foreign ownership and R&D investment exhibit an inverted U-shaped relationship because the incentives to monitor managers' decision-making processes initially play a greater role, but this role stagnates as the share owned by foreign investors becomes concentrated. In addition, we consider firm heterogeneity and observe the negative moderation effects of firm age and the positive moderation effects of growth opportunity.
    """
    
abstract_2 = """
    This study examined the role of perceived organizational commitment on managers' assessments of employees' career growth opportunities. Based on a paired sample of 161 legal secretaries and their managers, results indicated that managers used the attitudes and behaviors displayed by employees (strong extra-role performance and enhanced work engagement) as cues from which to base their perceptions of employees' affective commitment to the organization. In turn, employees perceived as highly committed to the organization experienced enhanced content and structural career growth opportunities. Moreover, the relation between managers' perceptions of employees' organizational commitment and content career growth opportunities was stronger for employees perceived as also highly committed to their careers than for employees perceived as less committed to their careers.
    """

abstract_3 = """
    This study examined how firms combine alliances and acquisitions in an exploration/exploitation framework. By conducting cluster analysis on a sample of 1270 acquisitions made by 836 firms, we first identified the patterns in alliance and acquisition activities undertaken by these firms. Five distinct patterns were identified: (I) low alliance-low acquisition, (II) low alliance-high acquisition, (III) high alliance-low acquisition, (IV) high alliance-high acquisition, and (V) medium alliance-very high acquisition. Next, we analyzed the different ways in which the two modes were interlinked within these five patterns for exploration/exploitation. Patterns III and IV appeared to involve both exploration/exploitation and mutually reinforce exploration/exploitation. In contrast, in the remaining patterns, the two modes appeared to be more loosely coupled with each other, with a focus on exploitation.
    """


abstracts = [
    abstract_0,
    abstract_1,
    abstract_2,
    abstract_3
]

# Define the chain components
method_extraction_prompt = PromptTemplate(
    template="{system_prompt}\n\nAbstract:\n{abstract}\n",
    input_variables=["system_prompt", "abstract"],
    partial_variables={"format_instructions": method_parser.get_format_instructions()}
)

abstract_analysis_prompt = PromptTemplate(
   template="{abstract_system_prompt}\n\n## Abstract: \n{abstract}\n",
   input_variables=["abstract_system_prompt", "abstract", "setence_analysis_json_example"],
   partial_variables={"format_instructions": abstract_parser.get_format_instructions()}
)

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
system_message_prompt = SystemMessagePromptTemplate.from_template(abstract_summary_system_template.template)
human_message_prompt = HumanMessagePromptTemplate.from_template("## Original Abstract: \n{abstract}")
chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])
chat_prompt.input_variables = [
    "method_json_format",
    "setence_analysis_json_example",
    "method_json_output", 
    "abstract_chain_output", 
    "abstract", 
    "json_structure"
]

def json_print_to_file(name, data):
    with open(f"{name}.json", "w") as f:
        json.dump(data, f, indent=4)

# # Define the chain
# summary_chain = (
#     RunnableParallel(
#         {
#             "method_json_output": method_extraction_prompt | llm | method_parser,
#             "abstract_chain_output": abstract_analysis_prompt | llm | abstract_parser,
#             "abstract": RunnablePassthrough.assign(abstract=lambda x: x["abstract"]),
#             "json_structure": RunnablePassthrough.assign(json_structure=lambda x: json_structure),
#             "method_json_format": RunnablePassthrough.assign(method_json_format=lambda x: method_json_format),
#             "setence_analysis_json_example": RunnablePassthrough.assign(setence_analysis_json_example=lambda x: setence_analysis_json_example)
#         }
#     )
#     | RunnablePassthrough.assign(
#         abstract_summary_system_prompt=lambda x: abstract_summary_system_template.format(
#             method_json_output=json.dumps(x["method_json_output"], indent=4),
#             abstract_chain_output=json.dumps(x["abstract_chain_output"], indent=4),
#             setence_analysis_json_example=setence_analysis_json_example,
#             method_json_format=method_json_format,
#             json_structure=json_structure
#         ),
#     )
#     | RunnablePassthrough.assign(
#         method_json_output=lambda x: (json_print_to_file("method_json_output", x["method_json_output"]), x["method_json_output"])[1],
#         abstract_chain_output=lambda x: (json_print_to_file("abstract_chain_output", x["abstract_chain_output"]), x["abstract_chain_output"])[1]
#     )
#     | chat_prompt
#     | llm
#     | abstract_summary_parser
# )

for i, abstract in enumerate(abstracts):
        # Define the chain
    summary_chain = (
        RunnableParallel(
            {
                "method_json_output": method_extraction_prompt | llm | method_parser,
                "abstract_chain_output": abstract_analysis_prompt | llm | abstract_parser,
                "abstract": RunnablePassthrough.assign(abstract=lambda x: x["abstract"]),
                "json_structure": RunnablePassthrough.assign(json_structure=lambda x: json_structure),
                "method_json_format": RunnablePassthrough.assign(method_json_format=lambda x: method_json_format),
                "setence_analysis_json_example": RunnablePassthrough.assign(setence_analysis_json_example=lambda x: setence_analysis_json_example)
            }
        )
        | RunnablePassthrough.assign(
            abstract_summary_system_prompt=lambda x: abstract_summary_system_template.format(
                method_json_output=json.dumps(x["method_json_output"], indent=4),
                abstract_chain_output=json.dumps(x["abstract_chain_output"], indent=4),
                setence_analysis_json_example=setence_analysis_json_example,
                method_json_format=method_json_format,
                json_structure=json_structure
            ),
        )
        | RunnablePassthrough.assign(
            method_json_output=lambda x: (json_print_to_file(f"method_json_output_{i}", x["method_json_output"]), x["method_json_output"])[1],
            abstract_chain_output=lambda x: (json_print_to_file(f"abstract_chain_output_{i}", x["abstract_chain_output"]), x["abstract_chain_output"])[1]
        )
        | chat_prompt
        | llm
        | abstract_summary_parser
    )
    # Invoke the chain
    try:
        summary_chain_output = summary_chain.invoke({
            "abstract": abstract,
            "system_prompt": system_prompt.content,
            "abstract_system_prompt": abstract_system_prompt.content,
            "setence_analysis_json_example": setence_analysis_json_example,
            "method_json_format": method_json_format,
            "json_structure": json_structure
        })
        print(json.dumps(summary_chain_output, indent=4))
    except Exception as e:
        print(f"Error: {type(e).__name__}: {str(e)}")
        import traceback
        traceback.print_exc()
        
    with open(f"summary_chain_output_{i}.json", "w") as f:
        json.dump(summary_chain_output, f, indent=4)